In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from textblob import Word
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Model
from keras.layers import Input, Dense, Embedding, Flatten, Conv1D, MaxPooling1D,Bidirectional,LSTM
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D,concatenate
from keras.activations import relu
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.preprocessing import text, sequence
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.optimizers import Adam
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from keras.preprocessing import text, sequence
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re
from nltk.corpus import stopwords
import os
print(os.listdir("../input"))

In [ ]:
print(os.listdir('../input/embeddings/'))

In [ ]:
EMBEDDING_FILE = ("../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec")

In [ ]:
train = pd.read_csv('../input/train.csv',encoding="utf-8")
test = pd.read_csv('../input/test.csv',encoding='utf-8')
submission = pd.read_csv('../input/sample_submission.csv')

In [ ]:
train['question_text'][58]

In [ ]:

train['question_text'] = train['question_text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
test['question_text'] = test['question_text'].apply(lambda x: " ".join(x.lower() for x in x.split()))

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]
def clean_text(x):

    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x


train["question_text"] = train["question_text"].apply(lambda x: clean_text(x))
test["question_text"] = test["question_text"].apply(lambda x: clean_text(x))



train['question_text'] = train['question_text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
test['question_text'] = test['question_text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))


train['question_text'] = train['question_text'].apply(lambda x: " ".join(x for x in x.split() if len(x) >2 ))
test['question_text'] = test['question_text'].apply(lambda x: " ".join(x for x in x.split() if len(x) >2 ))

"""stop = stopwords.words('english')


train['question_text'] = train['question_text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
test['question_text'] = test['question_text'].apply(lambda x: " ".join(x.lower() for x in x.split()))

train['question_text'] = train['question_text'].fillna("").apply(lambda x: clean_text(x))
test['question_text'] = test['question_text'].fillna("").apply(lambda x: clean_text(x))

train['question_text'] = train['question_text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
test['question_text'] = test['question_text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

train['question_text'] = train['question_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
test['question_text'] = test['question_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

train['question_text'] = train['question_text'].apply(lambda x: " ".join(x for x in x.split() if x.isalpha()))
test['question_text'] = test['question_text'].apply(lambda x: " ".join(x for x in x.split() if x.isalpha()))

train['question_text'] = train['question_text'].apply(lambda x: " ".join(x for x in x.split() if len(x) >2 ))
test['question_text'] = test['question_text'].apply(lambda x: " ".join(x for x in x.split() if len(x) >2 ))"""

In [ ]:
train['question_text'][58]

In [ ]:
#train['question_text'] = train['question_text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

#test['question_text'] = test['question_text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

In [ ]:
X_train = train['question_text']
Y_train = train['target']
X_test = test['question_text']


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
sequences_train = tokenizer.texts_to_sequences(X_train)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
max_input_lenght = max([len(x) for x in sequences_train])
max_input_lenght

In [ ]:
x_train_data_padded = pad_sequences(sequences_train, maxlen=max_input_lenght, padding='post', truncating='post')

In [ ]:
x_test_data_padded = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=max_input_lenght, padding='post', truncating='post')

In [ ]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
word_index = tokenizer.word_index
max_features = len(word_index)+1
def load_glove(word_index):
    EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if o.split(" ")[0] in word_index)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
            
    return embedding_matrix 
    
def load_fasttext(word_index):    
    EMBEDDING_FILE = '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100 and o.split(" ")[0] in word_index )

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector

    return embedding_matrix

def load_para(word_index):
    EMBEDDING_FILE = '../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100 and o.split(" ")[0] in word_index)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]
    
    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    
    return embedding_matrix

In [ ]:
import gc

embedding_matrix_1 = load_glove(word_index)
embedding_matrix_2 = load_fasttext(word_index)
#embedding_matrix_3 = load_para(word_index)
embedding_matrix = np.mean((embedding_matrix_1, embedding_matrix_2), axis=0)  
del embedding_matrix_1, embedding_matrix_2
gc.collect()
np.shape(embedding_matrix)

In [ ]:
from keras.layers import *

def squash(x, axis=-1):
    # s_squared_norm is really small
    # s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    # scale = K.sqrt(s_squared_norm)/ (0.5 + s_squared_norm)
    # return scale * x
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True)
    scale = K.sqrt(s_squared_norm + K.epsilon())
    return x / scale

# A Capsule Implement with Pure Keras
class Capsule(Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, kernel_size=(9, 1), share_weights=True,
                 activation='default', **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.kernel_size = kernel_size
        self.share_weights = share_weights
        if activation == 'default':
            self.activation = squash
        else:
            self.activation = Activation(activation)

    def build(self, input_shape):
        super(Capsule, self).build(input_shape)
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(1, input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     # shape=self.kernel_size,
                                     initializer='glorot_uniform',
                                     trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(input_num_capsule,
                                            input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     initializer='glorot_uniform',
                                     trainable=True)

    def call(self, u_vecs):
        if self.share_weights:
            u_hat_vecs = K.conv1d(u_vecs, self.W)
        else:
            u_hat_vecs = K.local_conv1d(u_vecs, self.W, [1], [1])

        batch_size = K.shape(u_vecs)[0]
        input_num_capsule = K.shape(u_vecs)[1]
        u_hat_vecs = K.reshape(u_hat_vecs, (batch_size, input_num_capsule,
                                            self.num_capsule, self.dim_capsule))
        u_hat_vecs = K.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))
        # final u_hat_vecs.shape = [None, num_capsule, input_num_capsule, dim_capsule]

        b = K.zeros_like(u_hat_vecs[:, :, :, 0])  # shape = [None, num_capsule, input_num_capsule]
        for i in range(self.routings):
            b = K.permute_dimensions(b, (0, 2, 1))  # shape = [None, input_num_capsule, num_capsule]
            c = K.softmax(b)
            c = K.permute_dimensions(c, (0, 2, 1))
            b = K.permute_dimensions(b, (0, 2, 1))
            outputs = self.activation(K.batch_dot(c, u_hat_vecs, [2, 2]))
            if i < self.routings - 1:
                b = K.batch_dot(outputs, u_hat_vecs, [2, 3])

        return outputs

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)

In [ ]:
embed_size=300
from keras.initializers import *

def capsule():
    K.clear_session()       
    inp = Input(shape=(max_input_lenght,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = SpatialDropout1D(rate=0.2)(x)
    x = Bidirectional(CuDNNGRU(100, return_sequences=True, 
                                kernel_initializer=glorot_normal(seed=12300), recurrent_initializer=orthogonal(gain=1.0, seed=10000)))(x)

    x = Capsule(num_capsule=10, dim_capsule=10, routings=4, share_weights=True)(x)
    x = Flatten()(x)

    x = Dense(100, activation="relu", kernel_initializer=glorot_normal(seed=12300))(x)
    x = Dropout(0.12)(x)
    x = BatchNormalization()(x)

    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer=Adam(),metrics=['accuracy',f1])
    return model

In [ ]:
model = capsule()

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train_data_padded, Y_train, train_size=0.98, random_state=233)

In [ ]:
from keras.callbacks import *
filepath="weights.best.hdf5"
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
#early = EarlyStopping(monitor="val_f1", mode="max", patience=2)
checkpoint = ModelCheckpoint(filepath, monitor='val_f1', verbose=2, save_best_only=True, mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_f1', factor=0.6, patience=1, min_lr=0.0001, verbose=2)
callbacks_list = [annealer,reduce_lr,checkpoint]
history = model.fit(x=x_train,y=y_train,batch_size=512, epochs=10,validation_split= 0.05,callbacks=callbacks_list)

In [ ]:
y_prediction1 = model.predict(x=x_val,batch_size=512)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
#for i in np.arange(0.1, 1, 0.01):
 #   print(i)
   # y_pred1 =(y_prediction1 > i).astype(int)
    #print(classification_report(y_val, y_pred1))


In [ ]:
y_prediction = model.predict(x=x_test_data_padded, batch_size=512)
y_pred = (y_prediction > 0.309).astype(int)
submission['prediction'] = y_pred
submission.to_csv('submission.csv', index=False)

In [ ]:
#submission[submission['prediction'] == 1]